In [1]:
import torch
from models.densenet import DenseNet
from models.MHA_resnet import ResidualNetwork
from models.resnet_bda import ResidualNetwork_classifier
from models.resnet_rnn import ResidualNetwork_lstm
from models.dense_rnn import  DenseNet_rnn
from models.dense_bda import DenseNet_bda
from models.dense_cam import DenseNet_cam
from models.dense_mha import DenseNet_mha
# from models.xresnet1d import *

In [2]:
# model1 = DenseNet(100,30).cuda()
# model1.load_state_dict(torch.load('model_weights/densenet-32-aur-0.964-auc 0.615 .pth'))


In [2]:
from easydict import EasyDict as edict
config = edict({'hidden_size':320, 'num_attention_heads':8})  
model2 = ResidualNetwork(12,config).cuda()
model2.load_state_dict(torch.load('model_weights/Multi_head_Arrythmia detection-13-aur-0.975-auc 0.62  (1).pth'), strict=False)

<All keys matched successfully>

In [9]:
model3 = ResidualNetwork_classifier(12).cuda()
model3.load_state_dict(torch.load('model_weights/RESNET_two_way-41-aur-0.972-auc 0.608 .pth'),strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['classifier.linear.weight', 'classifier.linear.bias', 'classifier.attention.fc1.weight', 'classifier.attention.fc1.bias', 'classifier.attention.fc2.weight', 'classifier.attention.fc2.bias'])

In [5]:
model4 = ResidualNetwork_lstm(12,bidirectional = False, rnn = 'gru').cuda()
model4.load_state_dict(torch.load('model_weights/F_CONV_GRU-24-aur-0.968-auc 0.64 .pth'))


<All keys matched successfully>

In [13]:
model5 = DenseNet_rnn(100,30).cuda()
model5.load_state_dict(torch.load('model_weights/dense-41-aur-0.97-auc 0.64 .pth'))

<All keys matched successfully>

In [7]:
# model6 = DenseNet_bda(100,30).cuda()
# model6.load_state_dict(torch.load('model_weights/dense_bda-1-aur-0.967-auc 0.594 .pth'),strict=False)

In [15]:
 model7 = DenseNet_cam(100,30).cuda()
model7.load_state_dict(torch.load('model_weights/dense_cam_17-aur-0.972-auc 0.623 .pth'))

<All keys matched successfully>

In [17]:
from easydict import EasyDict as edict
config = edict({'hidden_size':320, 'num_attention_heads':8})  
model8 = DenseNet_mha(100,30,config).cuda()
model8.load_state_dict(torch.load('model_weights/dense-24-aur-0.976-auc 0.634 .pth'), strict=False)

<All keys matched successfully>

In [3]:
import torch
import torchvision
import torch.nn as nn
import numpy as np, os, sys, joblib
import matplotlib.pyplot as pl
import pandas as pd
import random, os
import librosa
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
 #from torch.utils.data import random_split
from torch.optim import lr_scheduler
import time
import tqdm
from evaluate_model import *
from my_helper_code import *
from helper_code import *
#from model import *
from torch.nn import  Conv1d,Conv2d
from dataset import My_Dataset_separate

%matplotlib inline

In [4]:
def seed_everything(seed: int):
    
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(0)

In [5]:
df = pd.read_csv('dx_mapping_scored.csv')
labels = df['SNOMEDCTCode'].values
labels = [str(i) for i in labels]
classes = list(labels)

test_data_directory = 'test_data'
test_header_files, test_recording_files = find_challenge_files(test_data_directory)
test_num_recordings = len(test_recording_files)
training_classes = list(labels)
test_classes = list(labels)
num_classes = len(classes)
num_classes

30

In [6]:
sample_length = 4096
twelve_leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
test_dataset = My_Dataset_separate(test_header_files, test_recording_files, twelve_leads,sample_length,test_classes)


test_loader = DataLoader(dataset=test_dataset, batch_size=1,
                             shuffle=False, num_workers=0, pin_memory=True)

In [10]:

from evaluate_model import *
th = [.2,.3]

def one_zero(data,th):
    result = []
    for i,j in enumerate(data):
        if (data[i] > th) :
            result.append(1)
        else :
            result.append(0)
    return result

def generating_output_files(models,weight, test_classes,test_loader, output_directory,th) :
    for model in models :
        model.eval()
    for inputs, target, header_files in tqdm.tqdm(test_loader):
        header_files = header_files[0]        
        input_var = torch.autograd.Variable(inputs.cuda().float())
        target_var = torch.autograd.Variable(target.cuda().float())
        output = 0
        if weight :
            for i,model in enumerate(models):
                
                output = output+model(input_var)*(weight[i])
            output = output/sum(weight)
        else :
            output = models[0](input_var)
        
        probabilities = output.detach().cpu().numpy().squeeze()
        labels = one_zero(probabilities,th)
        header = load_header(header_files)
        recording_id = get_recording_id(header)
        head, tail = os.path.split(header_files)
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_directory, root + '.csv')
        save_outputs(output_file, recording_id, test_classes, labels, probabilities)
        
        
def test_model(model,weight, test_classes, test_loader, label_directory, output_directory,th) : 
    generating_output_files(model,weight, test_classes, test_loader, output_directory,th)
    classes, auroc, auprc, auroc_classes, auprc_classes, accuracy, f_measure, f_measure_classes, challenge_metric,cf_matrix = evaluate_model('test_data','test_outputs')
    print(f'Auroc : {auroc}')
    print(f'Accuracy : {accuracy}')
    print(f'f1 {f_measure}')
    print(f'challenge_metric{challenge_metric}')
    return	 classes, auroc, auprc, auroc_classes, auprc_classes, accuracy, f_measure, f_measure_classes, challenge_metric,cf_matrix
# tensor([[[0.9949],
#          [0.1105],
#          [0.9002],
#          [0.9921]]], device='cuda:0', grad_fn=<SigmoidBackward0>)   

In [19]:

weight = None
label_directory = 'test_data'
output_directory='test_outputs'
models = [model8]
cf_matrix_list = []
for i in th :
    print(f'for th {i}')
    classes, auroc, auprc, auroc_classes, auprc_classes, accuracy, f_measure, f_measure_classes, challenge_metric,cf_matrix = test_model(models,weight, test_classes, test_loader, label_directory, output_directory,i) 
    cf_matrix_list.append(cf_matrix)

  0%|                                                                                 | 2/4413 [00:00<04:40, 15.73it/s]

for th 0.2


100%|██████████████████████████████████████████████████████████████████████████████| 4413/4413 [02:16<00:00, 32.24it/s]


Loading weights...
Loading label and output files...
Evaluating model...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- Challenge metric...


  0%|                                                                                 | 2/4413 [00:00<04:27, 16.51it/s]

Done.
Auroc : 0.9760846512615529
Accuracy : 0.5563108996147745
f1 0.642042988659263
challenge_metric0.7373136450157349
for th 0.3


100%|██████████████████████████████████████████████████████████████████████████████| 4413/4413 [02:18<00:00, 31.85it/s]


Loading weights...
Loading label and output files...
Evaluating model...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- Challenge metric...
Done.
Auroc : 0.9760846512615529
Accuracy : 0.6025379560389758
f1 0.6498674132966807
challenge_metric0.7153049900471004


In [18]:
model = [model2, model3, model4, model5,model7,model8]
weight = [1,1,1,1,1,1]
label_directory = 'test_data'
output_directory='test_outputs'
for i in th:
    print(f'for th {i}')
    cf_matrix = test_model(model,weight, test_classes, test_loader, label_directory, output_directory,i) 

  0%|                                                                                         | 0/4413 [00:00<?, ?it/s]

for th 0.2


100%|██████████████████████████████████████████████████████████████████████████████| 4413/4413 [08:26<00:00,  8.72it/s]


Loading weights...
Loading label and output files...
Evaluating model...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- Challenge metric...


  0%|                                                                                         | 0/4413 [00:00<?, ?it/s]

...

Loading weights...
Loading label and output files...
Evaluating model...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- Challenge metric...


  0%|                                                                                         | 0/4413 [00:00<?, ?it/s]

Done.
Auroc : 0.9791910648836137
Accuracy : 0.6317697711307501
f1 0.6561949164910517
challenge_metric0.7281301900704895
for th 0.4


100%|██████████████████████████████████████████████████████████████████████████████| 4413/4413 [08:24<00:00,  8.75it/s]


Loading weights...
Loading label and output files...
Evaluating model...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- Challenge metric...
Done.
Auroc : 0.9791910648836137
Accuracy : 0.6501246317697711
f1 0.6374265369708496
challenge_metric0.6994859830000091


In [22]:
class ensemble(nn.Module):
    def __init__(self, n_models) :
        super(ensemble, self).__init__()
        self.n_models = n_models
        self.conv1d = nn.Conv1d(self.n_models,1,1)
        
    def forward(self,x) :
        return torch.sigmoid(self.conv1d(x)).squeeze()
def compute_accuracy(labels, outputs):
    num_recordings, num_classes = np.shape(labels)

    num_correct_recordings = 0
    for i in range(num_recordings):
        if np.all(labels[i, :]==outputs[i, :]):
            num_correct_recordings += 1

    return float(num_correct_recordings) / float(num_recordings)

In [ ]:
label_files, output_files = find_challenge_files_eval('test_data','test_outputs')

In [62]:
models = [model2, model3, model4, model5,model7]
num_epochs = 100
learning_rate = 0.001
for i in models:
    i.eval()
net = ensemble(5).cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
for epoch in range(0,num_epochs):
        since = time.time()
        print('Epoch {}/{}'.format(epoch , num_epochs - 1))
        print('-' * 10)
    
        net.train() 
        epoch_loss = 0.0
        epoch_acc = 0.0
        running_acc = 0.0
        running_loss = 0.0
        for i,(inputs, target, header_files) in tqdm.tqdm(enumerate(test_loader)):
                header_files = header_files[0]        
                input_var = torch.autograd.Variable(inputs.cuda().float())
                target_var = torch.autograd.Variable(target.cuda().float())
                output_list = []
                for model in models:
                        with torch.no_grad():
                            a = model(input_var)
                            if a.shape[0] ==30 :
                                output_list.append(a.unsqueeze(dim = 0))
                            else :
                                output_list.append(a)

                output = torch.cat(output_list,dim=0).unsqueeze(dim=0)
                result = net(output).unsqueeze(dim=0)
                
                loss = criterion(result, target_var)
                label = result.ge(0.5).float()
                np_label = label.detach().cpu().numpy()
                np_target = target_var.detach().cpu().numpy()
                if np.all(np_label[0,:]==np_target[0,:]):
                    epoch_acc += 1
                    running_acc +=1 
                    
                running_loss += loss.data.item()
                if (i%500) == 0:
                
                    print('step: {} totalloss: {loss:.3f} running_acc: {running_acc:.3f} '.format(i, loss = running_loss/500, running_acc = running_acc/500))
                    
                    running_acc = 0.0
                    running_loss = 0.0
        
                loss.backward() 
                optimizer.step()

        print('epoc acc: {epoch_acc:.3f}  '.format(epoch_acc = epoch_acc/4413) )      
        PATH = f'log/weight-{epoch}-acc-{round(epoch_acc,3)} .pth' 
        torch.save(model.state_dict(), PATH)
    

0it [00:00, ?it/s]

Epoch 0/99
----------


3it [00:00,  5.03it/s]

step: 0 totalloss: 0.001 running_acc: 0.000 


503it [00:39, 12.62it/s]

...

step: 3500 totalloss: 0.100 running_acc: 0.784 


4002it [06:07, 12.17it/s]

step: 4000 totalloss: 0.139 running_acc: 0.806 


4303it [06:37, 10.83it/s]


KeyboardInterrupt: 

In [ ]:
weights_file = 'weights.csv'
sinus_rhythm = set(['426783006'])
classes, weights = load_weights(weights_file)
labels = load_labels(label_files, classes)

binary_outputs, scalar_outputs = load_classifier_outputs(output_files, classes)
cf_matrix = compute_modified_confusion_matrix(labels,binary_outputs)

In [ ]:
for j in range(26) :
    cf_matrix[j,:] = cf_matrix[j,:]/sum(cf_matrix[j,:])

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(50,50))  
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
# ax.xaxis.set_ticklabels(['False])
# ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('save_as_a_png.png')
plt.show()


In [ ]:

import seaborn as sns
fig, ax = plt.subplots(figsize=(20,20))  
ax = sns.heatmap(weights, annot=True, cmap='Blues')

ax.set_title('Dx Scores\n\n');
ax.set_xlabel('\n Predicted Diseases');
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
# ax.xaxis.set_ticklabels(['False])
# ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('Dx scores.png')
plt.show()


In [ ]:
i) 